In [ ]:
from pyspark.sql import SparkSession

# spark = SparkSession\
#     .builder\
#     .appName("myApp")\
#     .config("spark.mongodb.input.uri", "mongodb://mongo:27017/yelp.startup_log")\
#     .config("spark.mongodb.output.uri", "mongodb://mongo:27017/local.startup_log")\
#     .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
#     .config("spark.sql.repl.eagerEval.enabled", True)\
#     .getOrCreate()
spark = SparkSession\
    .builder\
    .master("local[*]")\
    .appName("myApp")\
    .config("spark.mongodb.input.uri", "mongodb://192.168.0.86:27017/yelp.startup_log")\
    .config("spark.mongodb.output.uri", "mongodb://192.168.0.86:27017/local.startup_log")\
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
    .config("spark.sql.repl.eagerEval.enabled", True)\
    .getOrCreate()

In [ ]:
review_df = spark\
    .read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://192.168.0.86:27017/local.yelp")\
    .load()

business_df = spark\
    .read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://192.168.0.86:27017/local.business")\
    .load()


In [ ]:
# business_df.show(1)

In [ ]:
# review_df.show(1)

In [ ]:
# from pyspark.sql.functions import avg,count

# # stars cnt_views
# agg_rev_df = review_df.groupBy("business_id")\
#         .agg( \
#                 avg("stars").alias("mean_stars"), \
#                 count("*").alias("cnt_reviews")
#      )

# tmp = business_df.select("business_id","categories","postal_code")

# # catelist bid stars cnt_v

# agg_rev_df.join(tmp, on="business_id", how="inner").show(5)

# # review_df.groupBy("business_id").mean("stars").show(1)

In [ ]:
# business_df.schema

In [ ]:
# # business_df_nyc = business_df.filter( (business_df['postal_code']>"10000") & (business_df['postal_code']<"10163"))
# # business_df_nyc = business_df.filter( (business_df['postal_code'] == "10001") )
# business_df.groupBy("state").count().show(50)

In [ ]:
from pyspark.sql.functions import explode,split,avg,count,round,col

In [ ]:
# 各州评论总数

# tmp2 = business_df.select("business_id","state")
# tmp2 = tmp2.join(review_df, on="business_id",how="inner")\
#     .groupBy("state")\
#     .agg(\
#         count("*").alias("cnt_reviews"))

# tmp2.orderBy(col("cnt_reviews").desc())

In [ ]:


# business_df_pa = business_df.filter( (business_df['state'] == "PA") )

tmp = business_df.select("business_id","categories","state").withColumn('category_name',explode(split('categories',',')))

tmp


business_id,categories,state,category_name
Pns2l4eNsfO8kk83d...,"Doctors, Traditio...",CA,Doctors
Pns2l4eNsfO8kk83d...,"Doctors, Traditio...",CA,Traditional Chin...
Pns2l4eNsfO8kk83d...,"Doctors, Traditio...",CA,Naturopathic/Hol...
Pns2l4eNsfO8kk83d...,"Doctors, Traditio...",CA,Acupuncture
Pns2l4eNsfO8kk83d...,"Doctors, Traditio...",CA,Health & Medical
Pns2l4eNsfO8kk83d...,"Doctors, Traditio...",CA,Nutritionists
mpf3x-BjTdTEA3yCZ...,"Shipping Centers,...",MO,Shipping Centers
mpf3x-BjTdTEA3yCZ...,"Shipping Centers,...",MO,Local Services
mpf3x-BjTdTEA3yCZ...,"Shipping Centers,...",MO,Notaries
mpf3x-BjTdTEA3yCZ...,"Shipping Centers,...",MO,Mailbox Centers


In [ ]:

tmp = tmp.join(review_df, on="business_id",how="inner")\
    .groupBy("state", "category_name")\
    .agg(\
        round(avg("stars"), 2).alias("mean_stars"), \
        count("*").alias("cnt_reviews"))\

tmp

state,category_name,mean_stars,cnt_reviews
NJ,Car Wash,2.84,804
PA,Hot Pot,4.09,4559
FL,Endodontists,3.24,2025
IN,Home & Garden,3.37,1704
FL,Solar Installation,3.5,307
TN,Landmarks & Histo...,4.22,692
PA,Car Stereo Insta...,4.36,750
MO,Convenience Stores,3.43,267
NV,Real Estate Services,2.21,427
IL,Waxing,3.42,322


In [ ]:
tmp.count()

21861

In [ ]:
# from pyspark.sql.functions import col,bround

# tmp = tmp.select("postal_code","category_name",round(tmp["mean_stars"], 2).alias("avg_stars"), "cnt_reviews").orderBy(col("cnt_reviews").desc())
# tmp

In [ ]:
file_path =  "file:///D:/Program Files/PROJECT/out1"
tmp.coalesce(1).write.option("header","true").format("csv").save("out1")